In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime
import os
import collections
import time

In [2]:
client = pymongo.MongoClient()
db = client['Fields']
path = r'C:\Users\ChastLai\Documents\Data\Ticks\\'
tick_list = os.listdir(path)
data = pd.DataFrame()
price = pd.DataFrame(db['收盤價'].find({'日期': {'$gte': datetime.datetime(2022, 6, 1), '$lte': datetime.datetime.today()}}, {'_id': 0})).set_index('日期')
priceo = pd.DataFrame(db['開盤價'].find({'日期': {'$gte': datetime.datetime(2022, 6, 1), '$lte': datetime.datetime.today()}}, {'_id': 0})).set_index('日期')
price = (price / priceo - 1).shift(-1)
price.columns = price.columns.astype(int)

In [3]:
total_, iso_, nis_, nis_s_, nis_c_, nis_b_ = [], [], [], [], [], []
df_ = pd.DataFrame()
def calculator(df_, tick):
    ctime = time.time()
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    pf = price.loc[stk_test.index[0].strftime('%Y-%m-%d')]

    ms = 0.1
    total = {x: [0, 0] for x in stk_test.code.unique()}
    iso = {x: [0, 0] for x in stk_test.code.unique()}
    nis = {x: [0, 0] for x in stk_test.code.unique()}
    nis_s = {x: [0, 0] for x in stk_test.code.unique()}
    nis_c = {x: [0, 0] for x in stk_test.code.unique()}
    nis_b = {x: [0, 0] for x in stk_test.code.unique()}

    
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    for _, v in stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        if v['tick_type'] == 1:
            iso[v.code][0] += v['volume']
            total[v.code][0] += v['volume']
        elif v['tick_type'] == 2:
            iso[v.code][1] += v['volume']
            total[v.code][1] += v['volume']
    print(len(stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))
    for i, v in stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        time_package = [i - datetime.timedelta(milliseconds=ms), i + datetime.timedelta(milliseconds=ms)]
        nis_package = stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby(['ts', 'code']).sum()
        if len(nis_package) == 1:
            if v['tick_type'] == 1: # 外盤成交
                iso[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                iso[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
        elif len(nis_package.index.get_level_values(1).unique()) == 1:
            if v['tick_type'] == 1: # 外盤成交
                nis_s[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
                nis[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                nis_s[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
                nis[v.code][1] += v['volume']
        else:
            if len(nis_package[nis_package.index.get_level_values(1) == v['code']]) == 1:
                if v['tick_type'] == 1: # 外盤成交
                    nis_c[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_c[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
            else:
                if v['tick_type'] == 1: # 外盤成交
                    nis_b[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_b[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
    print(tick.split('.')[0])
    for k, l in {'total': total, 'iso': iso, 'nis': nis, 'nis_s': nis_s, 'nis_c': nis_c, 'nis_b': nis_b}.items():
        pf1 = pf.loc[l.keys()]
        pf1.name = 'returns'
        se = pd.Series({key: (v1[0]-v1[1]) / (v1[0]+v1[1]+100.99) if (v1[0] + v1[1]) != 0 else 0 for key, v1 in l.items()})
        se.name = 'coi'
        d = pd.concat([se, pf1], axis=1)#.corr().iloc[0, 1]
        d.sort_values('coi', ascending=False, inplace=True)
        d.reset_index(inplace=True)
        d = pd.concat([d, pd.Series([k]*len(d), name='Type')], axis=1)
        dh = d.iloc[:5, :].reset_index()
        dl = d.iloc[-5:, :].reset_index()
        dc = pd.concat([dh, dl], axis=1)
        df_ = pd.concat([df_, dc], axis=0)
        globals()[k + '_'].append(d.corr().iloc[1, 2])
        print(k, '{:.2f}'.format(d.corr().iloc[1, 2]))
    print('耗時: ', '{:.2f}'.format(time.time() - ctime))
    return df_
for tick in tick_list[-1:]:
    df_ = calculator(df_, tick)


2286
2022-11-23
total nan
iso nan
nis nan
nis_s nan
nis_c nan
nis_b nan
耗時:  24.32


In [7]:
# 加入回測
nis_s_ = []
nis_s_long = {}
nis_s_short = {}
df_, dh_, dl_ = pd.DataFrame(), [], []
def calculator(dh_, dl_, tick):
    if len(dh_) != 0:
        dhi = dh_
        dli = dl_
    ctime = time.time()
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    if len(dh_) != 0:
        for i in dhi:
            se_ = stk_test[stk_test['code'] == i]
            se_ = se_[se_['ask_price'] > 0]
            try:
                nis_s_long[tick.split('.')[0]] = [
                    se_['bid_price'].max() / se_['close'].head(1).iloc[0] - 1 ,
                    se_['close'].head(1).iloc[0] / se_['ask_price'].min() - 1,
                    se_['close'].tail(1).iloc[0] / se_['close'].head(1).iloc[0] - 1
                ]
            except Exception as e:
                print(e, se_['bid_price'].max(), se_['close'].head(1), se_['close'].tail(1), se_['close'].min())
                nis_s_long[tick.split('.')[0]] = [0, 0, 0]
        for i in dli:
            se_ = stk_test[stk_test['code'] == i]
            se_ = se_[se_['ask_price'] > 0]
            try:
                nis_s_short[tick.split('.')[0]] = [
                    se_['bid_price'].max() / se_['close'].head(1).iloc[0] - 1 ,
                    se_['close'].head(1).iloc[0] / se_['ask_price'].min() - 1, 
                    se_['close'].tail(1).iloc[0] / se_['close'].head(1).iloc[0] - 1
                ]
            except Exception as e:
                print(e, se_['bid_price'].max(), se_['close'].head(1), se_['close'].tail(1), se_['close'].min())
                nis_s_short[tick.split('.')[0]] = [0, 0, 0]            
    ms = 0.1
    nis_s = {x: [0, 0] for x in stk_test.code.unique()}

    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    for i, v in stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        time_package = [i - datetime.timedelta(milliseconds=ms), i + datetime.timedelta(milliseconds=ms)]
        nis_package = stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby(['ts', 'code']).sum()
        if len(nis_package) == 1:
            pass
        elif len(nis_package.index.get_level_values(1).unique()) == 1:
            if v['tick_type'] == 1: # 外盤成交
                nis_s[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                nis_s[v.code][1] += v['volume']
    k = 'nis_s'
    l = nis_s
    se = pd.Series({key: (v1[0]-v1[1]) / (v1[0]+v1[1]+100.99) if (v1[0] + v1[1]) != 0 else 0 for key, v1 in l.items()})
    se.name = 'coi'
    se.sort_values(ascending=False, inplace=True)
    dh = se.iloc[:5].index
    dl = se.iloc[-5:].index
    print('耗時: ', '{:.2f}'.format(time.time() - ctime))
    return dh, dl
for tick in tick_list[:]:
    dh_, dl_ = calculator(dh_, dl_, tick)


耗時:  11.64
耗時:  9.99
耗時:  14.19
耗時:  9.20
耗時:  7.62
耗時:  7.17
耗時:  6.32
耗時:  10.43
耗時:  11.73
耗時:  11.22
耗時:  12.70
耗時:  19.76
耗時:  17.77
耗時:  14.62
耗時:  29.63
耗時:  32.13
耗時:  19.34
耗時:  16.69
耗時:  25.65
耗時:  28.41
耗時:  28.04
耗時:  43.99
耗時:  35.97
耗時:  32.01
耗時:  31.03
耗時:  31.36
耗時:  28.05
耗時:  16.83
耗時:  22.01
耗時:  21.06
耗時:  21.53
耗時:  16.64
耗時:  17.07
耗時:  13.41
耗時:  21.07
耗時:  21.50
耗時:  35.85
耗時:  22.24
耗時:  17.55
耗時:  14.51
耗時:  17.63
耗時:  14.97
耗時:  11.47
耗時:  13.52
耗時:  11.25
耗時:  15.93
耗時:  11.93
耗時:  11.63
耗時:  9.59
耗時:  10.69
耗時:  10.71
耗時:  10.12
耗時:  19.90
耗時:  13.61
耗時:  11.38
耗時:  10.01
耗時:  11.42
耗時:  10.81
耗時:  9.46
耗時:  9.42
耗時:  10.52
耗時:  15.04
耗時:  14.11
耗時:  11.35
耗時:  12.56
耗時:  18.46
耗時:  19.58
耗時:  15.94
耗時:  13.53
single positional indexer is out-of-bounds nan Series([], Name: close, dtype: float64) Series([], Name: close, dtype: float64) nan
耗時:  11.32
耗時:  10.12
耗時:  11.52
耗時:  11.52
耗時:  13.55
耗時:  11.26
耗時:  10.26
耗時:  10.36
耗時:  9.39
耗時:  12.87
耗時:  14.8

In [9]:
nis_s_s = pd.DataFrame(nis_s_short, index=['H-O', 'O-L', 'C-O']).T
nis_s_l = pd.DataFrame(nis_s_long, index=['H-O', 'O-L', 'C-O']).T

In [21]:
nis_s_s.std()

H-O    0.014750
O-L    0.016063
C-O    0.021770
dtype: float64

In [4]:
df_.T.to_csv(r'C:\Users\ChastLai\Documents\Python Scripts\Output\PaperTrade\COI.csv')

In [5]:
long = df_.iloc[:, :5].dropna(how='all')
short = df_.iloc[:, 5:].dropna(how='all')
#pd.concat([df_.iloc[:, :4].dropna(), df_.iloc[:, 4:].dropna()], axis=1, ignore_index=True)

In [6]:
print(long[long['Type'] == 'nis_s'])
print(short[short['Type'] == 'nis_s'])


   level_0  index       coi  returns   Type
0        0   2888  0.840958      NaN  nis_s
1        1   2603  0.707861      NaN  nis_s
2        2   2891  0.681062      NaN  nis_s
3        3   2338  0.608589      NaN  nis_s
4        4   2882  0.597724      NaN  nis_s
   level_0  index       coi  returns   Type
0       41   2615 -0.376426      NaN  nis_s
1       42   3481 -0.494540      NaN  nis_s
2       43   5425 -0.570965      NaN  nis_s
3       44   2498 -0.658806      NaN  nis_s
4       45   1101 -0.821219      NaN  nis_s


In [24]:
long.columns = long.columns + '_long'
short.columns = short.columns + '_short'
long.reset_index(inplace=True)
short.reset_index(inplace=True)
ls = pd.concat([long, short], axis=1)
ls

,index,level_0_long,index_long,coi_long,returns_long,Type_long,index,level_0_short,index_short,coi_short,returns_short,Type_short
0,0,0,2344,0.342803,NaN,total,0,35,2409,-0.263978,NaN,total
1,1,1,3034,0.269363,NaN,total,1,36,8069,-0.271380,NaN,total
2,2,2,5483,0.268605,NaN,total,2,37,5425,-0.343323,NaN,total
3,3,3,2603,0.214911,NaN,total,3,38,2317,-0.435538,NaN,total
4,4,4,3037,0.207228,NaN,total,4,39,2882,-0.439249,NaN,total
5,0,0,2344,0.340177,NaN,iso,0,35,8069,-0.266264,NaN,iso
6,1,1,5483,0.271377,NaN,iso,1,36,2409,-0.268144,NaN,iso
7,2,2,3034,0.265550,NaN,iso,2,37,5425,-0.392109,NaN,iso
8,3,3,2603,0.218154,NaN,iso,3,38,2317,-0.435538,NaN,iso
9,4,4,6488,0.202304,NaN,iso,4,39,2882,-0.455606,NaN,iso


In [25]:
for t in ['total', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']:
    print(t, '{:.2f}'.format(ls[ls['Type_long'] == t].reset_index(drop=True).groupby(ls[ls['Type_long'] == t].reset_index(drop=True).index//5).sum().corr().iloc[3,2]))
    print(t, '{:.2f}'.format(ls[ls['Type_short'] == t].reset_index(drop=True).groupby(ls[ls['Type_short'] == t].reset_index(drop=True).index//5).sum().corr().iloc[-1,-2]))
    print('return: ', '{:.2f}'.format(ls[ls['Type_long'] == t].sum()['returns_long'] - ls[ls['Type_short'] == t].sum()['returns_short']))
#ls.groupby(ls.index//5).sum()

total nan
total nan
return:  0.00
iso nan
iso nan
return:  0.00
nis nan
nis nan
return:  0.00
nis_s nan
nis_s nan
return:  0.00
nis_c nan
nis_c nan
return:  0.00
nis_b nan
nis_b nan
return:  0.00


In [114]:
for t in ['total', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']:
    ls[ls['Type_long'] == 'nis_s'].sum()['returns_long'] - ls[ls['Type_short'] == 'nis_s'].sum()['returns_short']

0.44361125363971565

In [31]:
ms = 100
for tick in tick_list:
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    print(tick)
    print('iso_len: ', len(stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))
    print('nis_len: ', len(stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))

2022-09-01.json
iso_len:  192338
nis_len:  47572
2022-09-02.json
iso_len:  203630
nis_len:  53000
2022-09-05.json
iso_len:  159038
nis_len:  37177
2022-09-06.json
iso_len:  162272
nis_len:  33332
2022-09-07.json
iso_len:  153871
nis_len:  37283
2022-09-08.json
iso_len:  137825
nis_len:  32462
2022-09-12.json
iso_len:  147739
nis_len:  34889
2022-09-13.json
iso_len:  149964
nis_len:  36793
2022-09-14.json
iso_len:  172461
nis_len:  31745
2022-09-15.json
iso_len:  157423
nis_len:  30601
2022-09-16.json
iso_len:  150545
nis_len:  27736
2022-09-19.json
iso_len:  149136
nis_len:  29460
2022-09-20.json
iso_len:  164748
nis_len:  32943
2022-09-21.json
iso_len:  202163
nis_len:  79345
2022-09-22.json
iso_len:  197715
nis_len:  45887
2022-09-23.json
iso_len:  172529
nis_len:  37619
2022-09-26.json
iso_len:  194841
nis_len:  38171
2022-09-27.json
iso_len:  167639
nis_len:  34763
2022-09-28.json
iso_len:  206242
nis_len:  45836
2022-09-29.json
iso_len:  181904
nis_len:  43723
2022-09-30.json
iso_